In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\xgboost\__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [107]:
data = pd.read_csv('./datasets/titanic/train.csv', index_col = 'PassengerId')

In [108]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [109]:
for col in data.columns:
    if data[col].hasnans:
        nans = data[col].shape[0] - data.loc[data[col].notna()].shape[0]
        print('{} col has {} NaNs out of {} vals'.format(col, nans, data[col].shape[0]))

Age col has 177 NaNs out of 891 vals
Cabin col has 687 NaNs out of 891 vals
Embarked col has 2 NaNs out of 891 vals


In [110]:
data['Ticket'].nunique()

681

In [111]:
data.drop(columns = ['Name','Ticket','Cabin'], inplace = True)

In [112]:
to_category = ['Pclass','Sex', 'Embarked']
for col in to_category:
    data[col] = data[col].astype('category')
data['Survived'] = data['Survived'].astype('bool')

In [113]:
data = pd.get_dummies(data, columns = to_category, prefix = to_category)
data.drop(columns = ['Sex_male'], inplace = True)
data.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
1,False,22.0,1,0,7.2500,0,0,1,0,0,0,1
2,True,38.0,1,0,71.2833,1,0,0,1,1,0,0
3,True,26.0,0,0,7.9250,0,0,1,1,0,0,1
4,True,35.0,1,0,53.1000,1,0,0,1,0,0,1
5,False,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [114]:
X = data[[col for col in data.columns if col != 'Survived']]
y = data['Survived']
X.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,
1,22.0,1,0,7.2500,0,0,1,0,0,0,1
2,38.0,1,0,71.2833,1,0,0,1,1,0,0
3,26.0,0,0,7.9250,0,0,1,1,0,0,1
4,35.0,1,0,53.1000,1,0,0,1,0,0,1
5,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3)

In [116]:
median_age = X_train['Age'].median()
X_train.loc[:, 'Age'] = X_train['Age'].fillna(median_age)

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [117]:
X_proc = X.fillna(X.median())

In [118]:
make_pipeline(StandardScaler(), svm.SVC())

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [119]:
model = make_pipeline(StandardScaler(), svm.SVC())
param_grid = {'svc__kernel':['poly'],
             'svc__degree':[2,3,4,5],
             'svc__C':[0.1,1]}
search = GridSearchCV(model, param_grid, cv = 5)
search.fit(X_proc, y)
print('best score = {}'.format(search.best_score_))
search.best_params_

best score = 0.8215488215488216


{'svc__C': 1, 'svc__degree': 3, 'svc__kernel': 'poly'}

In [120]:
model = make_pipeline(StandardScaler(), svm.SVC())
param_grid = {'svc__kernel':['rbf'],
             'svc__gamma':[0.1, 1, 10],
             'svc__C':[0.1,1]}
search = GridSearchCV(model, param_grid, cv = 5)
search.fit(X_proc, y)
print('best score = {}'.format(search.best_score_))
search.best_params_

best score = 0.8215488215488216


{'svc__C': 1, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}

In [121]:
model = make_pipeline(StandardScaler(), svm.SVC(C = 1, kernel='poly', degree=3, gamma = 0.1))
scores = cross_val_score(model, X_proc, y, cv = 5)
print('Accuracy = {} +/- {}'.format(scores.mean(), scores.std() * 2))

Accuracy = 0.8193502913517936 +/- 0.05510745621593857


In [122]:
model = make_pipeline(StandardScaler(), RandomForestClassifier())
param_grid = {'randomforestclassifier__n_estimators':[10,20,30,40],
             'randomforestclassifier__max_depth':[2,3,4,5,6],
             'randomforestclassifier__criterion':['gini','entropy']}
search = GridSearchCV(model, param_grid, cv = 5)
search.fit(X_proc, y)
print('best score = {}'.format(search.best_score_))
search.best_params_

best score = 0.8226711560044894


{'randomforestclassifier__criterion': 'entropy',
 'randomforestclassifier__max_depth': 6,
 'randomforestclassifier__n_estimators': 20}

In [123]:
model = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 40, max_depth = 6))
scores = cross_val_score(model, X_proc, y, cv = 5)
print('Accuracy = {} +/- {}'.format(scores.mean(), scores.std() * 2))

Accuracy = 0.8137447970360883 +/- 0.03818522055309085


In [124]:
model.fit(X_proc, y)
model.steps[1][1].feature_importances_

array([0.12342663, 0.04672912, 0.04415399, 0.17172384, 0.04417153,
       0.02278602, 0.09670106, 0.40774387, 0.0125899 , 0.00924464,
       0.02072942])

In [125]:
feature_scores = np.array(model.steps[1][1].feature_importances_)
feature_scores[feature_scores > 0.1]

array([0.12342663, 0.17172384, 0.40774387])

In [126]:
model = make_pipeline(StandardScaler(), GradientBoostingClassifier())
param_grid = {'gradientboostingclassifier__n_estimators':[25, 50, 75, 100],
             'gradientboostingclassifier__max_depth':[2,3,4,5,6]}
search = GridSearchCV(model, param_grid, cv = 5)
search.fit(X_proc, y)
print('best score = {}'.format(search.best_score_))
search.best_params_

best score = 0.8338945005611672


{'gradientboostingclassifier__max_depth': 4,
 'gradientboostingclassifier__n_estimators': 50}

In [72]:
model = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators = 50, max_depth = 5))
scores = cross_val_score(model, X_proc, y, cv = 5)
print('Accuracy = {} +/- {}'.format(scores.mean(), scores.std() * 2))

Accuracy = 0.8351124393438228 +/- 0.05468137043035524


In [89]:
make_pipeline(StandardScaler(), xgb.XGBClassifier(max_depth = 5, reg_lambda = 1))

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbclassifier', XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constra...os_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None))])

In [93]:
model = make_pipeline(StandardScaler(), xgb.XGBClassifier(max_depth = 5, reg_lambda = 1))
param_grid = {'xgbclassifier__max_depth':[2,3,4,5],
             'xgbclassifier__reg_lambda':[0.1, 1, 10],
             'xgbclassifier__learning_rate':[0.1, 1, 10]}
search = GridSearchCV(model, param_grid, cv = 5)
search.fit(X_proc, y)
search.best_params_

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

...

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

{'xgbclassifier__learning_rate': 0.1,
 'xgbclassifier__max_depth': 5,
 'xgbclassifier__reg_lambda': 10}

In [131]:
model = make_pipeline(StandardScaler(), xgb.XGBClassifier(max_depth = 5, reg_lambda = 10, learning_rate = 0.1))
scores = cross_val_score(model, X_proc, y, cv = 5)
print('Accuracy = {} +/- {}'.format(scores.mean(), scores.std() * 2))

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sk

Accuracy = 0.8328272312766887 +/- 0.04432019815467643


C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [135]:
model = make_pipeline(StandardScaler(), xgb.XGBClassifier(max_depth = 5, reg_lambda = 10, learning_rate = 0.1))
model.fit(X_proc, y)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('xgbclassifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
  ...cale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None))])

In [136]:
test_data = pd.read_csv('./datasets/titanic/test.csv', index_col = 'PassengerId')

In [137]:
test_data.drop(columns = ['Name','Ticket','Cabin'], inplace = True)
to_category = ['Pclass','Sex', 'Embarked']
for col in to_category:
    test_data[col] = test_data[col].astype('category')
test_data = pd.get_dummies(test_data, columns = to_category, prefix = to_category)
test_data.drop(columns = ['Sex_male'], inplace = True)
test_data.loc[:, 'Age'] = test_data['Age'].fillna(median_age)
test_data.loc[:, 'Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [138]:
y_pred = model.predict(test_data)
y_pred = np.array(y_pred, dtype = 'uint8')

C:\Users\Lab716A-PC\AppData\Local\conda\conda\envs\py35\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
out = pd.DataFrame({'Survived':y_pred}, index = test_data.index)

In [139]:
out.shape

(418, 1)

In [140]:
out.to_csv('titanic_sub_xbg.csv')